In [1]:
import torch
from transformers import AutoTokenizer
from llama_model import *
import torch.nn.functional as F

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("pythainlp/thainer-corpus-v2-base-model")

In [3]:
#Load model
dim = 288
n_layers =  6
n_heads =  6
n_kv_heads = n_heads
multiple_of = 32
dropout = 0.1
max_seq_len = 350
model_args = ModelArgs(
    dim=dim,
    n_layers=n_layers,
    n_heads=n_heads,
    n_kv_heads=n_heads,
    vocab_size=tokenizer.vocab_size ,
    multiple_of=multiple_of,
    max_seq_len=max_seq_len,
    dropout=dropout,
) 
model = Transformer(model_args)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load('best_train_loss_model.pth'))
# model.load_state_dict(torch.load('best_val_loss_model.pth'))
model.to(device)
model.eval()

Transformer(
  (tok_embeddings): Embedding(25005, 288)
  (dropout): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0-5): 6 x TransformerBlock(
      (attention): Attention(
        (wq): Linear(in_features=288, out_features=288, bias=False)
        (wk): Linear(in_features=288, out_features=288, bias=False)
        (wv): Linear(in_features=288, out_features=288, bias=False)
        (wo): Linear(in_features=288, out_features=288, bias=False)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (feed_forward): FeedForward(
        (w1): Linear(in_features=288, out_features=768, bias=False)
        (w2): Linear(in_features=768, out_features=288, bias=False)
        (w3): Linear(in_features=288, out_features=768, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (attention_norm): RMSNorm()
      (ffn_norm): RMSNorm()
    )
  )
  (norm): RMSNorm()
  (output): Linear(in_features=288, ou

In [5]:
def generate_text_function(model, tokenizer, prompt, max_new_tokens=50, temperature=0.2, top_k=10,):

    tokenized_prompt =  tokenizer.encode(prompt)[1:-1]
    tokenized_prompt = (torch.tensor(tokenized_prompt, dtype=torch.long, device=device)[None, ...])

    generated_tokens = []
    context_tokens = tokenized_prompt
    for _ in range(max_new_tokens):

        context_tokens = context_tokens[:, -min(model.params.max_seq_len, context_tokens.size(1)):]

        output = model(context_tokens)
        logits = output[:, -1, :]
        logits = logits / temperature
        v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
        logits[logits < v[:, [-1]]] = -float('Inf')
        probs = F.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        context_tokens = torch.cat((context_tokens, next_token), dim=1)

        generated_tokens.append(next_token.item())

    # Decode the generated tokens to text
    generated_text = tokenizer.decode(generated_tokens)
    return generated_text


In [12]:
prompt = "เราสามารถจะใช้กูเกิ้ลค้นหาอะไร"
generated_text = generate_text_function(model, tokenizer, prompt)
print(generated_text)

สื่อข่าวคุณเจ้าหน้าการปกครองผลท่าทีท่าทีท่าทีท่าทีในการกรณีกรณีกระแสกรณีกระแสความรู้สึกกระแสความรู้สึกความรู้สึกความรู้สึกความรุนแรงเหตุการณ์เหตุการณ์เหตุการณ์เรื่องเรื่องเรื่องเรื่องเรื่องเรื่องเรื่องเพศเพศเพศแรงงานเพศเพศแรงงานเพศเพศแรงงานเพศท้องถิ่นเพศท้องถิ่นเพศท้องถิ่นปเพศเพศ


In [26]:
prompt = "ศาลอุทธรณ์"
generated_text = generate_text_function(model, tokenizer, prompt)
print(generated_text)

เรียกร้องคณะกรรมการ องค์กรปฏิรูป ปฏิรูปกฎหมายกฎหมายกฎหมายกฎหมาย กฎหมาย  พพพพ<unk>กพธ!&'ลางการยกเลิกการยกเลิกยศมหาวิทยาลัร้าว...........      พ. 


In [31]:
prompt = "เมืองหลวงของประเทศไทย"
generated_text = generate_text_function(model, tokenizer, prompt)
print(generated_text)

<unk>ประสบปัญหาก <unk> พ พ นพo พ พ...........      พ พ  "พ  " " " พ พ 


In [27]:
prompt = "ประเทศไทยใช้ภาษาอะไร"
generated_text = generate_text_function(model, tokenizer, prompt)
print(generated_text)

 ก <unk> พ เวลา นนพ ที่พ "พ  "พ.......     นพ พ พ.  พ พ..   " "
